# Bonus-Track Assignment 5: CharRNN, or “The Unreasonable Effectiveness of Recurrent Neural Networks”
## Character-level RNN for text generation


In [1]:
import numpy as np
import os
import requests
import random
import sys

import torch
from torch import cuda, zeros, Tensor, optim, nn
from torch.nn import Module, ModuleList, Sequential, Linear, Tanh, MSELoss, RNN

In [2]:
# Read a file and split into lines
url = 'https://www.gutenberg.org/files/100/100-0.txt' #iliade: 2199
filename = 'shakespeare.txt'
path = os.path.join(os.getcwd(), filename)

if not os.path.exists(path):
    response = requests.get(url)
    with open(path, 'wb') as text:
        text.write(response.content)

print(f'File downloaded and saved at: {path}')


File downloaded and saved at: c:\Users\Francesca\OneDrive\Desktop\cns_labs\computational-neuroscience\LAB3_2\bonus tracks\shakespeare.txt


In [3]:
text = open('./shakespeare.txt', "r", encoding='utf-8').read().lower()
# reduce it a bit (Inferno starts at character 2215)
text = text[60:196015]

print('corpus length:', len(text))


corpus length: 195955


In [4]:
maxlen = 60
step = 4

sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 48974
Unique characters: 59


In [5]:
y.shape, x.shape 

((48974, 59), (48974, 60, 59))

In [6]:
x = torch.tensor(x)
y = torch.tensor(y)
y.shape, x.shape 

(torch.Size([48974, 59]), torch.Size([48974, 60, 59]))

In [7]:
class charRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(charRNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
    

In [8]:
model = charRNN(x.shape[2], 128, x.shape[2])

In [9]:
def sample(preds, temperature=1.0):
    preds = preds.detach().numpy().astype('float64')  # Convert to numpy array
    preds = np.log(preds) / temperature  # Apply temperature scaling
    exp_preds = np.exp(preds)  # Exponentiate the scaled values
    preds = exp_preds / np.sum(exp_preds)  # Normalize to get probabilities
    probas = np.random.multinomial(1, preds, 1)  # Sample from the probability distribution
    return np.argmax(probas)  # Return the index of the highest probability


In [12]:

# Select a text seed at random
start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index:start_index + maxlen]
print('--- Generating text with seed: "' + generated_text + '"')

# Try a range of different temperatures
for temperature in [0.25, 0.5, 0.8, 1, 1.5]:
    print('----------------------- \nTemperature:', temperature)
    sys.stdout.write(generated_text)

    # Initialize the hidden state
    hidden = model.initHidden()

    # Generate 400 characters starting from the seed text
    for i in range(400):
        # One-hot encode the characters generated so far
        sampled = torch.zeros(1, len(chars))
        for char in generated_text:
            sampled[0, char_indices[char]] = 1.0

        # Forward pass through the model
        sampled = sampled.to(hidden.device)
        output, hidden = model.forward(sampled, hidden)

        # Sample the next character
        next_index = sample(output[0], temperature)
        next_char = chars[next_index]

        # Append the newly generated character
        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()
    sys.stdout.write('\n')

--- Generating text with seed: "fe in my ‘o lord, sir!’ i see things may
serve long, but not"
----------------------- 
Temperature: 0.25
fe in my ‘o lord, sir!’ i see things may
serve long, but nottensor([-4.2466, -4.0403, -4.3361, -4.1656, -3.9399, -3.9292, -4.0298, -3.8610,
        -4.1162, -4.1162, -4.1162, -4.1656, -4.1273, -4.0104, -4.0582, -3.9777,
        -4.0123, -4.1744, -4.1288, -4.0652, -4.4297, -4.1711, -3.8593, -4.2081,
        -4.1403, -4.1236, -4.0356, -4.2747, -3.9548, -4.1500, -4.2760, -4.0641,
        -3.9929, -4.0407, -4.2349, -4.1519, -4.1743, -4.2008, -3.9810, -4.2581,
        -4.0707, -3.7848, -3.8857, -4.0569, -4.1989, -4.1392, -3.9171, -3.9801,
        -4.1394, -3.9963, -3.9944, -4.0441, -3.9288, -4.3103, -4.1082, -4.2532,
        -3.9844, -4.0980, -3.8486], grad_fn=<SelectBackward>)


<ipython-input-9-4808b2b36866>:3: RuntimeWarning: invalid value encountered in log
  preds = np.log(preds) / temperature  # Apply temperature scaling


ValueError: pvals < 0, pvals > 1 or pvals contains NaNs